In [6]:
using DiffEqSensitivity, OrdinaryDiffEq, Zygote

function fiip(du,u,p,t)
  du[1] = dx = p[1]*u[1] - p[2]*u[1]*u[2]
  du[2] = dy = -p[3]*u[2] + p[4]*u[1]*u[2]
end
p = [1.5,1.0,3.0,1.0]; 
u0 = [1.0;1.0]
prob = ODEProblem(fiip,u0,(0.0,10.0),p)
alg = Rosenbrock23(autodiff=false)
sol = concrete_solve(prob,alg)

t: 84-element Array{Float64,1}:
  0.0
  0.0016779262355463645
  0.018457188591010008
  0.04592513806904994
  0.08861703360132858
  0.1459228411240901
  0.2218068620085969
  0.31568026828433493
  0.4346630038939746
  0.5830167748521553
  0.731370545810336
  0.8797243167685167
  1.0280780877266975
  ⋮
  8.786670745986017
  8.875563024627366
  8.980183220413753
  9.062226332288471
  9.155091351179243
  9.247956370070014
  9.357548531843364
  9.491295975399952
  9.629279390282264
  9.767262805164576
  9.905246220046887
 10.0
u: 84-element Array{Array{Float64,1},1}:
 [1.0, 1.0]
 [1.0008421283680495, 0.9966504744824827]
 [1.0096094419777581, 0.9638421465804013]
 [1.0253103275636828, 0.9127515548012217]
 [1.0529979220440533, 0.8394312875289703]
 [1.096399446156186, 0.7516968071859257]
 [1.164901929341672, 0.6522005194396727]
 [1.2674758984544852, 0.5515272397993951]
 [1.4274621095732376, 0.45292373012261483]
 [1.6787232083436345, 0.36524518624339086]
 [1.995685827977463, 0.30725774943401934]


In [25]:
@time du01,dp1 = Zygote.gradient((u0,p)->concrete_solve(prob,alg,u0,p,save_everystep=false,save_start=false)[1, end],u0,p)

  0.105094 seconds (262.47 k allocations: 9.111 MiB)


([1.9499410519050726, 0.19018844902536125], [2.1010803631070916, 0.17663895910776475, 0.544535424000591, 0.9371500287209719])

In [30]:
sol_adj = solve(prob,alg)

function dg(out,u, p, t, i)
  out .= [1.0, 0.0]
end
ts = sol_adj.t[end]
@time res = adjoint_sensitivities(sol_adj,Rosenbrock23(autodiff=false),dg,ts)

  0.434476 seconds (774.76 k allocations: 33.061 MiB)


([-1.9499410232907068, -0.19018846052941046], [-2.1010803174168005 -0.1766389659089034 -0.5445354222400056 -0.9371500062767397])

In [12]:
u1 = copy(u0)
p1 = copy(p)
h = 0.001
u1[1] = u1[1] + h
# p1[1] = p1[1] + h
prob1 = ODEProblem(fiip,u1,(0.0,10.0),p1)
sol1 = concrete_solve(prob1,alg,abstol=1e-10,reltol=1e-10)

du1_fd = (sol1[end] - sol[end])[1]/h

11.076917966749145